In [23]:
# clone codes
import os
os.chdir('/content/')
!git clone https://github.com/neuphonic/neutts-air
!apt-get install -y espeak
!pip install -r neutts-air/requirements.txt
!pip install llama-cpp-python onnxruntime
!apt-get install time ffmpeg
!/usr/bin/time -v
# other tts
!pip install edge-tts


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  time
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 46.7 kB of archives.
After this operation, 129 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 time amd64 1.9-0.1build2 [46.7 kB]
Fetched 46.7 kB in 1s (65.0 kB/s)
Selecting previously unselected package time.
(Reading database ... 127005 files and directories currently installed.)
Preparing to unpack .../time_1.9-0.1build2_amd64.deb ...
Unpacking time (1.9-0.1build2) ...
Setting up time (1.9-0.1build2) ...
Processing triggers for man-db (2.10.2-1) ...
/usr/bin/time: missing program to run
Try '/usr/bin/time --help' for more information.


In [9]:
# install perftest
!wget https://raw.githubusercontent.com/skiloop/myscripts/refs/heads/main/src/perftest/perftest
!mkdir /usr/local/bin
!install perftest /usr/local/bin
!chmod +x /usr/local/bin/perftest
!perftest -h

--2025-10-13 02:04:00--  https://raw.githubusercontent.com/skiloop/myscripts/refs/heads/main/src/perftest/perftest
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15186 (15K) [text/plain]
Saving to: ‘perftest.1’

perftest.1          100%[===================>]  14.83K  --.-KB/s    in 0.001s  

2025-10-13 02:04:00 (22.3 MB/s) - ‘perftest.1’ saved [15186/15186]

mkdir: cannot create directory ‘/usr/local/bin’: File exists
usage: perftest [-h] [-c COMMANDS] [-t TESTS] [-f CONFIG_FILE] [-n ITERATIONS]
                [-o OUTPUT_FILE] [-v] [--debug] [--no-color]
                [--sample-config SAMPLE_CONFIG]

性能测试工具 - 使用 /usr/bin/time 测试和比较多种命令的性能

options:
  -h, --help            show this help message and exit
  -c COMMANDS, --command COMMANDS
                 

In [39]:
# install and test whisper
!ls
!edge-tts --list-voices


examples  neuttsair  output.wav       perftest.json  requirements.txt
LICENSE   outputs    pefromance.json  README.md      samples
Name                                                Gender    ContentCategories    VoicePersonalities
--------------------------------------------------  --------  -------------------  ------------------------------------------------
af-ZA-AdriNeural                                    Female                         Well-Rounded, Animated, Bright
af-ZA-WillemNeural                                  Male
am-ET-AmehaNeural                                   Male
am-ET-MekdesNeural                                  Female
ar-AE-FatimaNeural                                  Female                         Crisp, Bright, Clear
ar-AE-HamdanNeural                                  Male
ar-BH-AliNeural                                     Male
ar-BH-LailaNeural                                   Female
ar-DZ-AminaNeural                                   Female             

In [34]:
# test neutts
# import os
# os.chdir('/content/neutts-air/')

# !python -m examples.onnx_example \
#   --input_text "My name is Dave, and um, I'm from London" \
#   --ref_codes samples/dave.pt \
#   --ref_text samples/dave.txt \
#   --backbone neuphonic/neutts-air-q4-gguf
# !python -m examples.basic_example \
#   --input_text "My name is Dave, and um, I'm from London" \
#   --ref_audio ./samples/dave.wav \
#   --ref_text ./samples/dave.txt \
#   --backbone neuphonic/neutts-air-q4-gguf
# !python -m examples.basic_example \
#   --input_text "My name is Dave, and um, I'm from London" \
#   --ref_audio ./samples/dave.wav \
#   --ref_text ./samples/dave.txt
# !python -m examples.onnx_example --input_text "My name is Dave, and um, I'm from London" --ref_audio ./samples/dave.wav --ref_text ./samples/dave.txt --ref_codes samples/dave.pt
# !whisper


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [30]:
# config performance test
import json

config = {
  "description": "TTS性能测试",
  "iterations": 5,
  "tests": [
    {
      "name": "neutts",
      "command": "python -m examples.basic_example --input_text \"My name is Dave, and um, I'm from London\" --ref_audio ./samples/dave.wav --ref_text ./samples/dave.txt"
    },
    {
      "name": "neutts-gguf",
      "command": "python -m examples.basic_example --input_text \"My name is Dave, and um, I'm from London\" --ref_audio ./samples/dave.wav --ref_text ./samples/dave.txt --backbone neuphonic/neutts-air-q4-gguf"
    },
    {
      "name": "neutts-onnx-gguf",
      "command": "python -m examples.onnx_example --input_text \"My name is Dave, and um, I'm from London\" --ref_text ./samples/dave.txt --ref_codes samples/dave.pt --backbone neuphonic/neutts-air-q4-gguf"
    },
    {
      "name": "neutts-onnx",
      "command": "python -m examples.onnx_example --input_text \"My name is Dave, and um, I'm from London\" --ref_text ./samples/dave.txt --ref_codes samples/dave.pt"
    },
    {
      "name": "edge-tts",
      "command": "edge-tts --text \"My name is Dave, and um, I'm from London\" --write-media ./outputs/dave-edge-tts.wav"
    },
  ]
}

with open('perftest.json', 'w') as f:
  json.dump(config, f)


In [31]:
# do performance test
!perftest -f perftest.json -o pefromance.json


性能测试工具
总共 4 个测试，每个运行 5 次


[1/4]
测试 [neutts]: python -m examples.basic_example --input_text "My name is Dave, and um, I'm from London" --ref_audio ./samples/dave.wav --ref_text ./samples/dave.txt
  完成 5 次运行

[2/4]
测试 [neutts-gguf]: python -m examples.basic_example --input_text "My name is Dave, and um, I'm from London" --ref_audio ./samples/dave.wav --ref_text ./samples/dave.txt --backbone neuphonic/neutts-air-q4-gguf
  完成 5 次运行

[3/4]
测试 [neutts-onnx-gguf]: python -m examples.onnx_example --input_text "My name is Dave, and um, I'm from London" --ref_text ./samples/dave.txt --ref_codes samples/dave.pt --backbone neuphonic/neutts-air-q4-gguf
  完成 5 次运行

[4/4]
测试 [neutts-onnx]: python -m examples.onnx_example --input_text "My name is Dave, and um, I'm from London" --ref_text ./samples/dave.txt --ref_codes samples/dave.pt
  完成 5 次运行

性能测试摘要

名称                   实际时间            用户时间            系统时间            相对速度      
---------------------------------------------------------------------